In [0]:
from pyspark.sql.functions import *

config = AppConfig(env)

df = spark.readStream \
    .option("kafka.bootstrap.servers", config.kafka_bootstrap) \
    ...

df_kafka = (
    spark.readStream
        .format("kafka")
        .option("kafka.bootstrap.servers", config.KAFKA_BOOTSTRAP)
        .option("subscribe", "orders_topic")
        .option("startingOffsets", "latest")
        .option("failOnDataLoss", "false")
        .load()
)

df_bronze = (
    df_kafka.selectExpr(
        "CAST(key AS STRING)",
        "CAST(value AS STRING)",
        "topic",
        "partition",
        "offset",
        "timestamp as kafka_timestamp"
    )
    .withColumn("ingestion_timestamp", current_timestamp())
    .withColumn("ingestion_date", current_date())
)

(
    df_bronze.writeStream
        .format("delta")
        .option("checkpointLocation", "abfss://checkpoints/bronze/orders")
        .outputMode("append")
        .trigger(processingTime="30 seconds")
        .toTable("prod.bronze.orders_raw")
)